In [1]:
import os
import re
import numpy as np
import pandas as pd

os.chdir('C:\\Users\\prowm')

Split della data in modo da ottenere giorno e mese separati

In [2]:
bookingPrice = pd.read_csv('HotelPrice_completo.csv', delimiter=';', header=0)
bookingPrice['Price'] = round(bookingPrice['Price'].astype(int), 0)

#Data divisa fra giorno, mese e anno
temp = bookingPrice['Check-in Date']
year = []; month = []; day = []
i = 0
for el in temp:
    spl = el.split('-')
    if len(spl) == 3:
        year.append(spl[0]); month.append(spl[1]); day.append(spl[2])
        i = i + 1
    else:
        print('Se printa questo è un problema')
        
bookingPrice = bookingPrice.assign(Year=year, Month=month, Day=day)
#bookingPrice.head()







trivagoPrice = pd.read_csv('HotelPriceTrivago.csv', delimiter=';', header=0)

#Cambio i prezzi da sterline in euro
print('Prezzo in sterline: %s' %(trivagoPrice['Price'].iloc[0]))
trivagoPrice['Price'] = round(trivagoPrice['Price'] * 1.12, 0)
trivagoPrice['Price'] = trivagoPrice['Price'].astype(int)

#Check
print('Prezzo in euro: %s' %(trivagoPrice['Price'].iloc[0]))

#Data divisa fra giorno, mese e anno
temp = trivagoPrice['Check-in Date']
year = []; month = []; day = []
i = 0
for el in temp:
    spl = el.split('-')
    if len(spl) == 3:
        year.append(spl[0]); month.append(spl[1]); day.append(spl[2])
        i = i +1
    else:
        print('Se printa questo è un problema')

trivagoPrice = trivagoPrice.assign(Year=year, Month=month, Day=day)
#trivagoPrice.head() 

Prezzo in sterline: 97.0
Prezzo in euro: 109


Si carica il dataset di match e si crea un dizionario chiave-valore con il nome originale e il nome modificato per il match

In [3]:
#Nomi di match
hotel = pd.read_csv('MatchingHotel.csv', delimiter=';', header=0)
#hotel.head()

diz = {}
l = len(hotel)
for i in range(l):
    if hotel['ID'].iloc[i] not in diz.keys():
        diz[hotel['ID'].iloc[i]] = hotel['Match'].iloc[i]

Si controllano i nomi in comune nei dataset dei prezzi

In [4]:
#Aggiornamento del dataset di Booking
temp = []
for el in bookingPrice['ID']:
    el_temp = el.split('_')
    el_temp[1] = el_temp[1].lower()
    el = el_temp[0] + '_' + el_temp[1]
    temp.append(el)
bookingPrice = bookingPrice.assign(newId=temp)


bookingMatch = []
l = len(bookingPrice)
for i in range(l):
    if bookingPrice['newId'].iloc[i] in diz.keys():
        bookingMatch.append(diz[bookingPrice['newId'].iloc[i]])
    else:
        bookingMatch.append('')
bookingPrice = bookingPrice.assign(Match=bookingMatch)



#Aggiornamento del dataset di Trivago
temp = []
for el in trivagoPrice['hotel_id']:
    el_temp = el.split('_')
    el_temp[1] = el_temp[1].lower()
    el = el_temp[0] + '_' + el_temp[1]
    temp.append(el)
trivagoPrice = trivagoPrice.assign(newId=temp)


trivagogMatch = []
l = len(trivagoPrice)
for i in range(l):
    if trivagoPrice['newId'].iloc[i] in diz.keys():
        trivagogMatch.append(diz[trivagoPrice['newId'].iloc[i]])
    else:
        trivagogMatch.append('')
trivagoPrice = trivagoPrice.assign(Match=trivagogMatch)

In [5]:
bookingPrice = bookingPrice[bookingPrice.Match != '']
trivagoPrice = trivagoPrice[trivagoPrice.Match != '']

Dato che diversi giorni singoli non corrispondono si prendono le settimane come criterio di match per hotel-data.

In [6]:
temp = []
for i in range(len(bookingPrice)):
    if int(bookingPrice['Day'].iloc[i]) < 8:
        temp.append(1)
    elif int(bookingPrice['Day'].iloc[i]) < 15:
        temp.append(8)
    elif int(bookingPrice['Day'].iloc[i]) < 22:
        temp.append(15)
    elif int(bookingPrice['Day'].iloc[i]) < 29:
        temp.append(22)
    else:
        temp.append(29)
bookingPrice = bookingPrice.assign(NewDay=temp)
        
    
temp = []
for i in range(len(trivagoPrice)):
    if int(trivagoPrice['Day'].iloc[i]) < 8:
        temp.append(1)
    elif int(trivagoPrice['Day'].iloc[i]) < 15:
        temp.append(8)
    elif int(trivagoPrice['Day'].iloc[i]) < 22:
        temp.append(15)
    elif int(trivagoPrice['Day'].iloc[i]) < 29:
        temp.append(22)
    else:
        temp.append(29)
trivagoPrice = trivagoPrice.assign(NewDay=temp)

Si creano dei nuovi id per fare il match hotel-data

In [7]:
temp = []
l = []
for i in range(len(bookingPrice)):
    if bookingPrice['Match'].iloc[i] in l:
        a = bookingPrice['Match'].iloc[i] + '_' + bookingPrice['Year'].iloc[i] + bookingPrice['Month'].iloc[i] + str(bookingPrice['NewDay'].iloc[i])
        if a not in temp:
            temp.append(a)
        else:
            temp.append('')
            pass
    elif bookingPrice['Match'].iloc[i] in sorted(trivagoPrice['Match']):
        l.append(bookingPrice['Match'].iloc[i])
        temp.append(bookingPrice['Match'].iloc[i] + '_' + bookingPrice['Year'].iloc[i] + bookingPrice['Month'].iloc[i] + str(bookingPrice['NewDay'].iloc[i]))
    else:
        temp.append('')
    if i%500 == 0:
        #print(i/len(bookingPrice))
        pass
        
temp2 = []
l = []
for i in range(len(trivagoPrice)):
    if trivagoPrice['Match'].iloc[i] in l:
        a = trivagoPrice['Match'].iloc[i] + '_' + trivagoPrice['Year'].iloc[i] + trivagoPrice['Month'].iloc[i] + str(trivagoPrice['NewDay'].iloc[i])
        if a not in temp2:
            temp2.append(trivagoPrice['Match'].iloc[i] + '_' + trivagoPrice['Year'].iloc[i] + trivagoPrice['Month'].iloc[i] + str(trivagoPrice['NewDay'].iloc[i]))
        else:
            temp2.append('')
            pass
    elif trivagoPrice['Match'].iloc[i] in sorted(bookingPrice['Match']):
        temp2.append(trivagoPrice['Match'].iloc[i] + '_' + trivagoPrice['Year'].iloc[i] + trivagoPrice['Month'].iloc[i] + str(trivagoPrice['NewDay'].iloc[i]))
        l.append(trivagoPrice['Match'].iloc[i])
    else:
        temp2.append('')
    if i%1000 == 0:
        #print(i/len(trivagoPrice))
        pass
    
print(len(temp), len(temp2))

21729 57102


In [8]:
#Aggiornamento
bookingPrice = bookingPrice.assign(FinalMatch=temp)
trivagoPrice = trivagoPrice.assign(FinalMatch=temp2)

bookingPrice2 = bookingPrice[bookingPrice.FinalMatch != '']
trivagoPrice2 = trivagoPrice[trivagoPrice.FinalMatch != '']
print(len(bookingPrice2), len(trivagoPrice2))

9534 19928


In [9]:
price = pd.merge(bookingPrice2, trivagoPrice2, how='left', left_on='FinalMatch', right_on='FinalMatch')

In [15]:
price = price.dropna()
temp = []
not_month = ['07', '08']
for i in range(len(price)):
    if price['Month_x'].iloc[i] not in not_month:
        temp.append(price['Price_x'].iloc[i]-price['Price_y'].iloc[i])
i = 0
somma = 0
for i in range(len(temp)):
    somma = somma + temp[i]
print(somma)
print(somma/len(temp))

453283.0
319.663610719


In [20]:
print(sorted(temp))

[-414.0, -79.0, -58.0, 44.0, 48.0, 78.0, 86.0, 87.0, 88.0, 91.0, 93.0, 95.0, 95.0, 95.0, 96.0, 97.0, 98.0, 99.0, 100.0, 101.0, 102.0, 102.0, 102.0, 102.0, 103.0, 103.0, 103.0, 105.0, 105.0, 106.0, 106.0, 106.0, 106.0, 106.0, 106.0, 106.0, 106.0, 106.0, 106.0, 107.0, 107.0, 108.0, 109.0, 109.0, 110.0, 110.0, 111.0, 111.0, 111.0, 112.0, 112.0, 112.0, 112.0, 113.0, 113.0, 113.0, 115.0, 116.0, 116.0, 117.0, 117.0, 117.0, 117.0, 118.0, 118.0, 118.0, 119.0, 120.0, 120.0, 120.0, 121.0, 121.0, 121.0, 122.0, 122.0, 122.0, 122.0, 123.0, 123.0, 123.0, 124.0, 124.0, 124.0, 124.0, 125.0, 125.0, 125.0, 125.0, 126.0, 126.0, 126.0, 126.0, 127.0, 127.0, 127.0, 127.0, 127.0, 127.0, 127.0, 128.0, 128.0, 128.0, 129.0, 129.0, 130.0, 130.0, 130.0, 131.0, 131.0, 131.0, 131.0, 131.0, 132.0, 132.0, 132.0, 132.0, 133.0, 133.0, 133.0, 134.0, 134.0, 134.0, 134.0, 134.0, 134.0, 134.0, 134.0, 134.0, 135.0, 136.0, 136.0, 136.0, 136.0, 136.0, 136.0, 137.0, 137.0, 137.0, 137.0, 138.0, 138.0, 139.0, 139.0, 139.0, 139.0

In [23]:
price = price.dropna()
temp = 0
for i in range(len(price)):
    if (price['Price_x'].iloc[i] - price['Price_y'].iloc[i]) < 0:
        temp = temp + 1
        print(price['Month_x'].iloc[i], price['ID'].iloc[i] + '\n' + str(price['Price_x'].iloc[i]) + ' vs ' + str(price['Price_y'].iloc[i]))
        print(price['Stars'].iloc[i])
print(temp)

09 Holiday Inn Express Manchester - Salford Quays_Manchester
257 vs 671.0
3.0
01 ibis Styles London Excel_London
198 vs 256.0
3.0
05 Holiday Inn Express Newcastle City Centre_Newcastle upon Tyne
247 vs 326.0
3.0
3
